In [1]:
import pandas as pd
import numpy as np
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='Wourly', api_key='ynOus3rUfBXLzn11FNII')
import matplotlib

In [2]:
#variable "ocean" is a Dataframe used in this code

#loading ocean
ocean = pd.read_csv("ocean.csv")
#general simplifying of ocean
ocean.drop(["Unnamed: 0"], axis = 1, inplace = True)
#low occurence of data for age lower than 16 and higher than 55, there are 10 or more participants for each age
ocean.drop(ocean.index[ocean['age'] <= 15].tolist(), inplace = True)
ocean.drop(ocean.index[ocean['age'] >= 56].tolist(), inplace = True)
ocean.dropna(axis = 0, how='any', thresh=None, subset=None, inplace = True)
#creating mean of individual questions
ocean['A'] = ocean[['A1', 'A2', 'A3', 'A4', 'A5']].mean(axis = 1)
ocean['C'] = ocean[['C1', 'C2', 'C3', 'C4', 'C5']].mean(axis = 1)
ocean['E'] = ocean[['E1', 'E2', 'E3', 'E4', 'E5']].mean(axis = 1)
ocean['N'] = ocean[['N1', 'N2', 'N3', 'N4', 'N5']].mean(axis = 1)
ocean['O'] = ocean[['O1', 'O2', 'O3', 'O4', 'O5']].mean(axis = 1)
#reorganization - cutting of unmeaned questions and resorting meaned ones
ocean.drop(['A1', 'A2', 'A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'C5', 'E1', 'E2', 'E3', 'E4', 'E5', 'N1', 'N2', 'N3', 'N4', 'N5', 'O1', 'O2', 'O3', 'O4', 'O5'], axis = 1, inplace = True)
ocean = ocean[['A', 'C', 'E', 'N', 'O', "age", "education", "gender"]].sort_values('age')
#removing values below scale
ocean = ocean[ocean['A'] > 1]
ocean = ocean[ocean['C'] > 1]
ocean = ocean[ocean['E'] > 1]
ocean = ocean[ocean['N'] > 1]
ocean = ocean[ocean['O'] > 1]
#removing decimals from 'education' column
ocean['education'] = ocean['education'].astype(int)
#final cut
ocean = ocean.reset_index(drop=True)

In [3]:
ocean.head(5)

,A,C,E,N,O,age,education,gender
0,3.6,4.2,3.6,5.2,3.4,16,3,2
1,4.6,4.8,3.6,2.4,3.6,16,1,2
2,3.0,4.4,4.0,4.0,3.6,16,3,2
3,4.0,4.4,4.4,4.2,4.0,16,3,2
4,4.2,4.6,4.2,4.2,4.2,16,1,1


In [4]:
#meaner() creates means in dimensions according 
def meaner(dimension, parameter):

    start = 0
    iteration = 0
    mean = []
    
    if parameter == "a":
        end = ocean['age'].value_counts().sort_index().values.tolist()[0]
        step = ocean['age'].value_counts().sort_index().values.tolist()
    elif parameter == "e":
        end = ocean.sort_values("education")["education"].value_counts().sort_index().values.tolist()[0]
        step = ocean.sort_values("education")["education"].value_counts().sort_index().values.tolist()
    elif parameter == "g":
        end = ocean.sort_values("gender")["gender"].value_counts().sort_index().values.tolist()[0]
        step = ocean.sort_values("gender")["gender"].value_counts().sort_index().values.tolist()
    
    if dimension == "A":
        column = 0
    elif dimension == "C":
        column = 1
    elif dimension == "E":
        column = 2
    elif dimension == "N":
        column = 3
    else:
        column = 4

    for distance in step:    
    
        if iteration == 0:
            mean.append(ocean.iloc[start:end, column].mean())
            iteration += 1
            start += distance
    
        else:
            end += distance
            mean.append(ocean.iloc[start:end, column].mean())
            iteration += 1
            start += distance
            
    return [round(number, 2) for number in mean]

In [5]:
lake_age = pd.DataFrame()
lake_age['age'] = ocean['age'].value_counts().sort_index().index.values.tolist()
lake_age['respondents'] = ocean['age'].value_counts().sort_index().values.tolist()
lake_age['A'] = meaner("A", "a")
lake_age['C'] = meaner("C", "a")
lake_age['E'] = meaner("E", "a")
lake_age['N'] = meaner("N", "a")
lake_age['O'] = meaner("O", "a")

lake_age.head()

,age,respondents,A,C,E,N,O
0,16,14,3.90,3.87,3.80,3.61,3.91
1,17,31,4.05,3.97,3.70,3.16,4.05
2,18,104,4.10,3.85,3.88,3.36,3.96
3,19,161,4.11,3.85,3.84,3.28,3.95
4,20,175,4.20,3.80,3.79,3.16,3.84


In [6]:
lake_edu = pd.DataFrame()
lake_edu['education'] = ocean.sort_values("education")["education"].value_counts().sort_index().index.values.tolist()
lake_edu['respondents'] = ocean.sort_values("education")["education"].value_counts().sort_index().values.tolist()
lake_edu['A'] = meaner("A", "e")
lake_edu['C'] = meaner("C", "e")
lake_edu['E'] = meaner("E", "e")
lake_edu['N'] = meaner("N", "e")
lake_edu['O'] = meaner("O", "e")

lake_edu.head()

,education,respondents,A,C,E,N,O
0,1,186,4.04,3.88,3.81,3.34,3.96
1,2,234,4.17,3.81,3.83,3.22,3.92
2,3,1032,4.24,3.82,3.81,3.28,3.89
3,4,324,4.28,3.81,3.77,3.20,3.82
4,5,336,4.30,3.81,3.72,3.01,3.85


In [7]:
lake_gen = pd.DataFrame()
lake_gen['gender'] = ocean.sort_values("gender")["gender"].value_counts().sort_index().index.values.tolist()
lake_gen['respondents'] = ocean.sort_values("gender")["gender"].value_counts().sort_index().values.tolist()
lake_gen['A'] = meaner("A", "g")
lake_gen['C'] = meaner("C", "g")
lake_gen['E'] = meaner("g", "g")
lake_gen['N'] = meaner("N", "g")
lake_gen['O'] = meaner("O", "g")

lake_gen.head()

,gender,respondents,A,C,E,N,O
0,1,680,4.14,3.85,3.89,3.24,3.89
1,2,1432,4.27,3.81,3.88,3.21,3.88


In [8]:
# Create a trace
trace_ocean = go.Box(
    name = "Respondents",
    x = ocean["age"],
    y = ocean["A"],

)

trace_lake_age = go.Scatter(
    name = "Mean",
    x = lake_age["age"],
    y = lake_age["A"],

)

layout = go.Layout(
    xaxis=dict(
        range=[15, 55],
        showgrid=True,
        zeroline=True,
        zerolinecolor='#969696',
        zerolinewidth=4


    ),
    yaxis=dict(
        range=[1, 6],
        showgrid=True,
        gridcolor='#bdbdbd',
        gridwidth=1
    )
)

data = [trace_ocean, trace_lake_age]

figure = go.Figure(data = data, layout = layout)

# Plot and embed in ipython notebook!
py.iplot(figure, filename = "age")

# or plot with: plot_url = py.plot(data, filename='basic-line')